### Import all necessary libraries

In [1]:
import yfinance as yf
import pandas as pd
import datetime
import calendar
import numpy as np
import matplotlib.pyplot as plt
from calendar import monthrange
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Get stock data from Yahoo Finance
I have chosen my favourite stock here Bajaj Finance. It is a high beta stocks and thus provides opportunites for short term trading.
Each entry represents 1 day staring from 1 Jan 2011

In [2]:
ticker='BAJFINANCE.NS'
tickerData = yf.Ticker(ticker)
df = tickerData.history(period='1d', start='2011-1-1')

### Bollinger Bands
Bollinger Bands are a technical analysis tool developed by John Bollinger for generating oversold or overbought signals.
There are three lines that compose Bollinger Bands: A simple moving average (middle band) and an upper and lower band.
The upper and lower bands are typically 2 standard deviations +/- from a 20-day simple moving average, but can be modified.

In [3]:
### Bollinger Bands
window = 21
no_of_std = 2

#Calculate rolling mean and standard deviation using number of days set above
rolling_mean = df['Close'].rolling(window).mean()
rolling_std = df['Close'].rolling(window).std()
rolling_vol = df['Volume'].rolling(window).mean()

#create two new DataFrame columns to hold values of upper and lower Bollinger bands
df['Rolling Mean'] = rolling_mean
df['Bollinger High'] = rolling_mean + (rolling_std * no_of_std)
df['Bollinger Low'] = rolling_mean - (rolling_std * no_of_std)
df['Rolling Vol'] = rolling_vol

### Relative Strength Index (RSI)
The relative strength index (RSI) is a momentum indicator used in technical analysis that measures the magnitude of recent price changes to evaluate overbought or oversold conditions in the price of a stock or other asset. The RSI is displayed as an oscillator (a line graph that moves between two extremes) and can have a reading from 0 to 100. The indicator was originally developed by J. Welles Wilder Jr. and introduced in his seminal 1978 book, "New Concepts in Technical Trading Systems."

Traditional interpretation and usage of the RSI are that values of 70 or above indicate that a security is becoming overbought or overvalued and may be primed for a trend reversal or corrective pullback in price. An RSI reading of 30 or below indicates an oversold or undervalued condition.

In [4]:
#### RSI
window = 14
no_of_std = 2
df['Close_shift'] = df['Close'].shift(30)
df['GL'] = (df['Close_shift']-df['Close'])/df['Close']
df['rsi']=None
for row in range(df.shape[0]):
    tmp = df['GL'].iloc[row-14:row]
    avg_gain = (tmp[tmp>0].mean()*13+df['GL'].iloc[row])/14
    avg_loss = (abs(tmp[tmp<0].mean())*13+df['GL'].iloc[row])/14
    rs = avg_gain/avg_loss
    df['rsi'].iloc[row] = 100-100/(1+rs)

### MACD
Moving Average Convergence Divergence (MACD) is a trend-following momentum indicator that shows the relationship between two moving averages of a security’s price. The MACD is calculated by subtracting the 26-period Exponential Moving Average (EMA) from the 12-period EMA.

The result of that calculation is the MACD line. A nine-day EMA of the MACD called the "signal line," is then plotted on top of the MACD line, which can function as a trigger for buy and sell signals. Traders may buy the security when the MACD crosses above its signal line and sell—or short—the security when the MACD crosses below the signal line. Moving Average Convergence Divergence (MACD) indicators can be interpreted in several ways, but the more common methods are crossovers, divergences, and rapid rises/falls.

In [5]:
#### macd
df['macd'] = df['Close'].ewm(span=12, adjust=False).mean()-df['Close'].ewm(span=26, adjust=False).mean()
df['signal'] = df['macd'].ewm(span=9, adjust=False).mean()
df['macd_hist'] = df['macd']-df['signal']

### Target variable
Buy today only if stock moves upwards tomorrow

In [6]:
df['Buy'] = df['GL']
df['Buy'] = df['Buy']>0.02
df['Buy']=df['Buy'].apply(lambda x:int(x))

In [7]:
def prev_weekday(d, weekday):
    days_ahead = d.weekday() - weekday
    if days_ahead <= 0:
        days_ahead += 7
    return d - datetime.timedelta(days_ahead)
def days2expiry(dt):
    yr = dt.year
    mth = dt.month
    d = datetime.date(yr,mth,monthrange(yr, mth)[1])
    expiry = prev_weekday(d, 3)
    return (expiry-dt.date()).days

### Feature Engineering
Using ideas from all of the above, let us develop some features.

In [8]:
#### Bollinger band features
df['BreakHigh'] = 0
df['BreakClose'] = 0
df['BreakHigh'][df['High']>df['Bollinger High']] = 1
df['BreakClose'][df['Close']>df['Bollinger High']] = 1

df['BreakLow'] = 0
df['BreakBear'] = 0
df['BreakLow'][df['Low']<df['Bollinger Low']] = 1
df['BreakBear'][df['Close']<df['Bollinger Low']] = 1

#### MACD features
df['MACDmoreSignal'] = 0
df['MACDmoreSignal'][df['macd']>df['signal']] = 1

#### RSI features
df['overbought']=0
df['overbought'][df['rsi']>70]=1
df['oversold']=0
df['oversold'][df['rsi']<30]=1

### Volume based features
df['OperatorEntry']=0
df['OperatorEntry'][df['Volume']>df['Rolling Vol']]=1

## HmO, HmC, HmL, OmL, CmL
df['HmO'] = (df['High'] - df['Open'])/df['Open']
df['HmC'] = (df['High'] - df['Close'])/df['Open']
df['HmL'] = (df['High'] - df['Low'])/df['Open']
df['OmL'] = (df['Open'] - df['Low'])/df['Open']
df['CmL'] = (df['Close'] - df['Low'])/df['Open']

### Simple moving average features
### SMAs 20, 50, 100, 200
df['sma_20'] = df['Close'].rolling(20).mean()
df['sma_50'] = df['Close'].rolling(50).mean()
df['sma_100'] = df['Close'].rolling(100).mean()
df['sma_200'] = df['Close'].rolling(200).mean()
df['20g50'] = df['sma_20'] > df['sma_50']
df['20g100'] = df['sma_20'] > df['sma_100']
df['20g200'] = df['sma_20'] > df['sma_200']
df['Cg20'] = df['Close'] > df['sma_20']
df['Cg50'] = df['Close'] > df['sma_50']
df['Cg100'] = df['Close'] > df['sma_100']
df['Cg200'] = df['Close'] > df['sma_200']

### DOW - Day of Week, DFME - Days from Monthly Expiry, DOM - Day of Month
df['date'] = df.index
df['dow'] = df.index.dayofweek
df['DFME'] = df['date'].apply(lambda x:days2expiry(x))
df['DOM'] = df['date'].apply(lambda x:x.day)

### Remove raw data and keep only useful features

In [9]:
df1=df[['macd_hist', 'Buy', 'BreakHigh', 'BreakClose',
       'BreakLow', 'BreakBear', 'MACDmoreSignal', 'overbought', 'oversold',
       'OperatorEntry', 'HmO', 'HmC', 'HmL', 'OmL', 'CmL', '20g50', '20g100',
       '20g200','dow', 'DFME', 'DOM', 'Cg20','Cg50','Cg100','Cg200']]
df1=df1.dropna()

### Autokeras
Let's test Autokeras library to get the best model

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf

import autokeras as ak

In [11]:
tt_split_days = 2000
col = [k for k in df.columns if k != 'Buy']
X_train = df[col].iloc[:tt_split_days]
y_train = df['Buy'].iloc[:tt_split_days]
X_test = df[col].iloc[tt_split_days:]
y_test = df['Buy'].iloc[tt_split_days:]

In [12]:
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(
    overwrite=True, max_trials=3, metrics='accuracy'
)  # It tries 3 different models.
# Feed the structured data classifier with training data.
clf.fit(
    X_train, y_train, epochs=10
)

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.85326087474823

Best val_accuracy So Far: 0.85326087474823
Total elapsed time: 00h 00m 13s
INFO:tensorflow:Oracle triggered exit
Epoch 1/10
63/63 [==============================] - 1s 2ms/step - loss: 0.5426 - accuracy: 0.7585
Epoch 2/10
63/63 [==============================] - 0s 2ms/step - loss: 0.3626 - accuracy: 0.8550
Epoch 3/10
63/63 [==============================] - 0s 2ms/step - loss: 0.2808 - accuracy: 0.8880
Epoch 4/10
63/63 [==============================] - 0s 2ms/step - loss: 0.2377 - accuracy: 0.9000
Epoch 5/10
63/63 [==============================] - 0s 2ms/step - loss: 0.2118 - accuracy: 0.9150
Epoch 6/10
63/63 [==============================] - 0s 2ms/step - loss: 0.1968 - accuracy: 0.9125
Epoch 7/10
63/63 [==============================] - 0s 2ms/step - loss: 0.1753 - accuracy: 0.9285
Epoch 8/10
63/63 [==============================] - 0s 2ms/step - loss: 0.1616 - accuracy: 0.9340
Epoch 9/10
63/63 [======================

In [13]:
# Predict with the best model.
predicted_y = clf.predict(X_test)

19/19 [==============================] - 0s 1ms/step


In [14]:
y_pred = [int(k[0]) for k in predicted_y]

In [15]:
y_test = [k for k in y_test]

In [16]:
from sklearn.metrics import confusion_matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [17]:
p = tp/(tp+fp)
r = tp/(tp+fn)
f1 = 2*p*r/(p+r)

In [18]:
p

0.9230769230769231

In [19]:
r

0.7397260273972602

In [20]:
f1

0.8212927756653992

In [21]:
X_test.shape

(584, 45)

In [22]:
X_train.shape

(2000, 45)

In [23]:
df.shape

(2584, 46)

### Returns when using this strategy

In [28]:
sum(df.iloc[2000:,:]['GL']*y_pred)

37.43763820334614

### God Returns

In [29]:
sum(df.iloc[2000:,:]['GL']*y_test)

39.956576393890074

### Buy and Hold Returns

In [36]:
(df.iloc[2000:,:]['Close'][-1]-df.iloc[2000:,:]['Close'][0])*100/df.iloc[2000:,:]['Close'][0]

129.62396696466536